# Berechnung einer absoluten Positionierung mit Code-Messungen, Teil 2

In [28]:
%%capture
# Requirements 
# pip install git+https://github.com/GNSSpy-Project/gnsspy
# pip install pyunpack
# pip install georinex

# Libs
import gnsspy as gp
import numpy as np
import georinex
import geopandas
import matplotlib.pyplot as plt
import math
import pandas as pd

# Params
np.set_printoptions(formatter={'float': '{: 0.5f}'.format})
plt.rcParams['figure.dpi'] = 300

In [29]:
%%capture
# Dataimport
station = gp.read_obsFile("./data/ONSA0320.11O")
ephemerides = georinex.load("./data/G3_11032.PRE")
clock = gp.read_clockFile("./data/cod16212.clk")

In [30]:
# get dfs for each epoch
clock_epoch_0 = clock[clock.Epoch == "2011-02-01 00:00:00"]
clock_epoch_1 = clock[clock.Epoch == "2011-02-01 00:15:00"]
clock_epoch_2 = clock[clock.Epoch == "2011-02-01 00:30:00"]
clock_epoch_3 = clock[clock.Epoch == "2011-02-01 00:45:00"]
clock_epoch_4 = clock[clock.Epoch == "2011-02-01 01:00:00"]

epochs_clock = list((clock_epoch_0, clock_epoch_1, clock_epoch_2, clock_epoch_3, clock_epoch_4))

ephemerides_epoch_0 = ephemerides.sel(time="2011-02-01T00:00:00.000000000")
ephemerides_epoch_1 = ephemerides.sel(time="2011-02-01T00:15:00.000000000")
ephemerides_epoch_2 = ephemerides.sel(time="2011-02-01T00:30:00.000000000")
ephemerides_epoch_3 = ephemerides.sel(time="2011-02-01T00:45:00.000000000")
ephemerides_epoch_4 = ephemerides.sel(time="2011-02-01T01:00:00.000000000")

epochs_ephemerides = list((ephemerides_epoch_0, ephemerides_epoch_1, ephemerides_epoch_2, ephemerides_epoch_3, ephemerides_epoch_4))


In [31]:
# Helper funcs for satellite Positions
# Consts
omega_e = 7.292115e-5 #s^-1
c = 299792458 #m/s

def calculateSatPos(earth_fixed_coords, sat_velocities):
    earth_fixed_coords_si = earth_fixed_coords * 1000 # km to m
    sat_velocities_si = sat_velocities / 10 # dm/s to m/s
    
    tau = math.dist(np.array(station.approx_position), coord) / c
    sat_coords = np.array(earth_fixed_coords).T - np.array( (sat_velocities_si + (omega_e * np.array([-earth_fixed_coords[1], earth_fixed_coords[0], 0]))) * tau)

    return sat_coords

# Helper funcs for tropospherical correction
# Rotation matrices
def ry(a): return np.matrix([[np.cos(a), 0, -np.sin(a)], [0, 1, 0], [np.sin(a), 0, np.cos(a)]])
def rz(a): return np.matrix([[np.cos(a), np.sin(a), 0], [-np.sin(a), np.cos(a), 0], [0, 0, 1]])

def calculateLatLong(earth_fixed_coords):
    x,y,z = earth_fixed_coords

    lat = math.degrees(math.atan2(z, math.sqrt(x**2 + y**2)))
    lon = math.degrees(math.atan2(y, x))

    return lat, lon

# Topo Coords
def calculateTopoCoords(stat_coord_earth_fixed, earth_fixed_coords_at_send):
    lat_s, lon_s = calculateLatLong(stat_coord_earth_fixed)
    r2 = ry((math.pi / 2) - lat_s)
    r3 = rz(lon_s)

    # Calculate N, E, U
    n, e, u = list(), list(), list()
    for coords in sat_coords_at_send:
        topo_coords = r2 @ r3 @ (coords - station.approx_position).T

        n.append(-topo_coords[0,0])
        e.append(topo_coords[0,1])
        u.append(topo_coords[0,2])

# Zenitwinkel
def calculateZn(topo_coords):
    n, e, u = topo_coords
    return math.degrees(math.atan2(math.sqrt(n[i]**2 + e[i]**2), u[i]))

# tropo delay
def calculateTropDelay(angle):
    return 2.4 / math.cos(math.radians(angle))

# Relativistic delay
def calculateRelativistics(coord, velocity):
    velocity /= 10
    return 2 * (coord @ velocity.T) / c

In [59]:
# Calculate Sat positions for all satellites in Interval A
sat_coords_at_send = list()
for epoch in epochs_ephemerides: # for each epoch
    epoch_helper = list()
    for j, coord in enumerate(epoch.position): # for each coordinate
        #print("koord", coord, "velo", epoch.velocity[j])
        epoch_helper.append(calculateSatPos(coord, epoch.velocity[j])) # calculate sat position at send time
    sat_coords_at_send.append(np.array(epoch_helper))

#sat_coords_at_send[0][24] * 1000
epochs_ephemerides[0].position[24]

<xarray.DataArray 'position' (ECEF: 3)>
array([ 18619.36024, -15884.79349,  10299.91528])
Coordinates:
    time     datetime64[ns] 2011-02-01
    sv       <U3 'G25'
  * ECEF     (ECEF) <U1 'x' 'y' 'z'